In [1]:
from src.draft_email_agent import generate_response
from src.draft_email_agent import extract_pdf_text
# Example usage
email = """
Dear Sales Team,

I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

Looking forward to hearing from you.

Best regards, 
John Doe
"""

product_catalog_pdf = 'company_docs\Product Catalog for Tech Solutions Ltd.pdf'
product_catalog_text = extract_pdf_text(product_catalog_pdf)
# get_action_from_response(determine_action(email))
# Generate the response
response = generate_response(email, product_catalog_text)
print(response)

[SystemMessage(content="\n            You are an assistant for a technical sales team. Your task is to read a customer's email and, using the provided product catalog, generate a detailed response.\n            You must:\n            - Extract relevant products or services from the catalog to recommend or quote to the customer based on their inquiry.\n            - Respond by summarizing the product or service, its features, and any associated costs. \n            If the email contains a request for a quote, provide the quote details. Otherwise, respond with product recommendations or explanations.\n            \n            The product catalog is provided below.\n            You are also given a customer email. Use both the catalog and the email content to craft a response.\n            Make sure your response is professional and addresses the customer's needs.\n            ", additional_kwargs={}, response_metadata={}), HumanMessage(content="Product Catalog:\nProduct Catalog for Tech

In [4]:
from src.ai.make_quote.master_quote_functions import process_document
from src.ui.app_config_functions import retrieve_relevant_context

email_text = """
    Dear Sales Team,

    I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

    Looking forward to hearing from you.

    Best regards, 
    John Doe
"""

compan_conx = retrieve_relevant_context("dev_team", "company_context", email_text)
user_context = ""
file_url = "https://ofgixvnkftwqcfbhjzdb.supabase.co/storage/v1/object/public/veloflow-company-docs/dev_team/company_docs/Tech%20Solutions%20Quote%20Template%20Ltd.pdf"

updated = process_document(file_url,
                           email_text,
                           compan_conx,
                           "",
                           "os@so.com")

[INFO] HTTP Request: GET https://ofgixvnkftwqcfbhjzdb.supabase.co/rest/v1/document_embeddings?select=id%2C%20company%2C%20type%2C%20filename%2C%20text%2C%20embedding&company=eq.dev_team&type=eq.company_context "HTTP/1.1 200 OK"
[INFO] Start to convert C:\Users\olive\AppData\Local\Temp\tmpfquh_g6_.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/2) Page 1
[INFO] (2/2) Page 2
[INFO] [4/4] Creating pages...
[INFO] (1/2) Page 1
[INFO] (2/2) Page 2
[INFO] Terminated in 1.93s.
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


GPT-4 response: ```json
[
    {
        "location": {
            "type": "paragraph",
            "index": 3
        },
        "original": "Date: [Insert Date] \nQuote Number: [Insert Quote Number] ",
        "replacement": "Date: 2025-03-24 \nQuote Number: [Insert Quote Number] "
    },
    {
        "location": {
            "type": "paragraph",
            "index": 4
        },
        "original": "[Customer Name]",
        "replacement": "John Doe"
    },
    {
        "location": {
            "type": "paragraph",
            "index": 4
        },
        "original": "[Customer Email]",
        "replacement": "os@so.com"
    },
    {
        "location": {
            "type": "table",
            "table_index": 0,
            "row_index": 1,
            "col_index": 2
        },
        "original": "[Insert ",
        "replacement": "1"
    },
    {
        "location": {
            "type": "table",
            "table_index": 0,
            "row_index": 1,
            "col_index"

In [6]:
output_file = "final_quote.docx"  # Change filename as needed
updated.save(output_file)
print(f"Document saved as {output_file}")

Document saved as final_quote.docx


In [8]:
import os
import pdfkit
import markdown
import pandas as pd
import pypandoc
from docx import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def convert_to_pdf(input_file, output_pdf):
    """
    Converts a given file (docx, html, tex, md, csv, xlsx) into a PDF.
    """
    file_ext = os.path.splitext(input_file)[1].lower()
    
    if file_ext == ".docx":
        doc = Document(input_file)
        text = "\n".join([p.text for p in doc.paragraphs])
        create_pdf_from_text(text, output_pdf)

    elif file_ext == ".html":
        pdfkit.from_file(input_file, output_pdf)

    elif file_ext == ".md":
        html_content = markdown.markdown(open(input_file, encoding="utf-8").read())
        pdfkit.from_string(html_content, output_pdf)

    elif file_ext == ".tex":
        os.system(f"pdflatex -output-directory={os.path.dirname(output_pdf)} {input_file}")

    elif file_ext in [".csv", ".xlsx"]:
        df = pd.read_csv(input_file) if file_ext == ".csv" else pd.read_excel(input_file)
        create_pdf_from_text(df.to_string(), output_pdf)

    else:
        raise ValueError("Unsupported file type.")

    print(f"PDF saved as: {output_pdf}")

def create_pdf_from_text(text, output_pdf):
    """
    Helper function to create a PDF from raw text.
    """
    c = canvas.Canvas(output_pdf, pagesize=letter)
    c.drawString(100, 750, text[:1000])  # Limits text to avoid overflow
    c.save()

# Example usage:
convert_to_pdf("final_quote.docx", "output.pdf")


PDF saved as: output.pdf


In [3]:
import os
import tempfile
import requests
from pdf2docx import Converter
from docx import Document
from docx2pdf import convert as docx2pdf_convert
from src.ai.make_quote.docx import *
# Assume these functions are already implemented for DOCX editing:
# extract_doc_structure(doc)
# get_replacements_from_gpt4(doc_structure, email_text, compan_conx, user_context)
# replace_text_in_doc(doc, replacements)

def convert_pdf_to_docx(pdf_path, docx_path):
    """
    Converts a PDF file to DOCX format using pdf2docx.
    """
    cv = Converter(pdf_path)
    cv.convert(docx_path, start=0, end=None)
    cv.close()

def process_pdf_document(file_url, email_text, compan_conx, user_context, user_email):
    """
    Downloads a PDF from a URL, converts it to DOCX, processes it for updates,
    and converts the result back to PDF.
    
    Returns the path to the updated PDF.
    """
    # Download the PDF to a temporary file
    response = requests.get(file_url)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {response.status_code}")
    
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as pdf_tmp:
        pdf_tmp.write(response.content)
        pdf_tmp_path = pdf_tmp.name

    # Convert PDF to DOCX
    with tempfile.NamedTemporaryFile(delete=False, suffix=".docx") as docx_tmp:
        docx_tmp_path = docx_tmp.name
    convert_pdf_to_docx(pdf_tmp_path, docx_tmp_path)

    # Process the DOCX file using your existing functions
    doc = Document(docx_tmp_path)
    doc_structure = extract_doc_structure_docx(doc)
    replacements = get_replacements_from_gpt_docx(doc_structure, email_text, compan_conx, user_context, user_email)
    print("Detected replacements:", replacements)
    updated_doc = replace_text_in_docx(doc, replacements)
    
    # Save updated DOCX to a new temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".docx") as updated_docx_tmp:
        updated_docx_path = updated_docx_tmp.name
    updated_doc.save(updated_docx_path)

    # Convert the updated DOCX back to PDF
    # We'll save the PDF to another temporary file
    updated_pdf_path = os.path.join(tempfile.gettempdir(), "final_quote.pdf")
    docx2pdf_convert(updated_docx_path, updated_pdf_path)

    # Clean up the intermediate temporary files if desired
    os.remove(pdf_tmp_path)
    os.remove(docx_tmp_path)
    os.remove(updated_docx_path)

    return updated_pdf_path


file_url = "https://ofgixvnkftwqcfbhjzdb.supabase.co/storage/v1/object/public/veloflow-company-docs/dev_team/company_docs/Tech%20Solutions%20Quote%20Template%20Ltd.pdf"

from src.ui.app_config_functions import retrieve_relevant_context

email_text = """
    Dear Sales Team,

    I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

    Looking forward to hearing from you.

    Best regards, 
    John Doe
"""

compan_conx = retrieve_relevant_context("dev_team", "company_context", email_text)
user_context = ""

updated_pdf = process_pdf_document(file_url, email_text, compan_conx, user_context, "os")
print("Updated PDF saved to:", updated_pdf)


[INFO] HTTP Request: GET https://ofgixvnkftwqcfbhjzdb.supabase.co/rest/v1/document_embeddings?select=id%2C%20company%2C%20type%2C%20filename%2C%20text%2C%20embedding&company=eq.dev_team&type=eq.company_context "HTTP/1.1 200 OK"
[INFO] Start to convert C:\Users\olive\AppData\Local\Temp\tmp2m83_1bs.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/2) Page 1
[INFO] (2/2) Page 2
[INFO] [4/4] Creating pages...
[INFO] (1/2) Page 1
[INFO] (2/2) Page 2
[INFO] Terminated in 1.43s.
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


GPT-4 response: ```json
[
    {
        "location": {
            "type": "paragraph",
            "index": 3
        },
        "original": "Date: [Insert Date] \nQuote Number: [Insert Quote Number] ",
        "replacement": "Date: 2025-03-24 \nQuote Number: [Insert Quote Number] "
    },
    {
        "location": {
            "type": "paragraph",
            "index": 4
        },
        "original": "To: \n[Customer Name] \n[Customer Company Name] \n[Customer Address] \n[Customer Email] \n[Customer Phone Number] ",
        "replacement": "To: \nJohn Doe \n[Customer Company Name] \n[Customer Address] \nos \n[Customer Phone Number] "
    },
    {
        "location": {
            "type": "table",
            "table_index": 0,
            "row_index": 1,
            "col_index": 2
        },
        "original": "[Insert ",
        "replacement": "1"
    },
    {
        "location": {
            "type": "table",
            "table_index": 0,
            "row_index": 1,
            "col

  0%|          | 0/1 [00:00<?, ?it/s]

Updated PDF saved to: C:\Users\olive\AppData\Local\Temp\final_quote.pdf


In [2]:
users = '{"oliver.a.simmonds@gmail.com": "dev_team", "abdi212121@gmail.com": "dev_team", "andygregory17@hotmail.com": "golf_breaks", "agregory@golfbreaks.com": "golf_breaks"}'
import json
users = json.loads(users)
print(users)



{'oliver.a.simmonds@gmail.com': 'dev_team', 'abdi212121@gmail.com': 'dev_team', 'andygregory17@hotmail.com': 'golf_breaks', 'agregory@golfbreaks.com': 'golf_breaks'}


In [2]:
x = "kdgsld?"
x.rstrip('?')

'kdgsld'